# First Step - Scraping

For the scrapping part I'll use Beatiful Soap.


In [1]:
# libraries to import
import pandas as pd
import numpy as np
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup

Necessary steps to convert a html page into a dataframe.

In [2]:
# scrape the data from the webpage to a table with 3 columns
PageToScrape = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
PageToScrape = BeautifulSoup(PageToScrape, 'lxml')
PageToScrape = PageToScrape.find('table',{'class':'wikitable sortable'})
PageToScrape = PageToScrape.find_all('tr')
#PageToScrape

table = []

for row in PageToScrape:
    table.append([t.text.strip() for t in row.find_all('td')])

    
df = pd.DataFrame(table, columns=['PostalCode', 'Borough', 'Neighbourhood'])
print(df.head(5))
print(df.shape)

  PostalCode       Borough     Neighbourhood
0       None          None              None
1        M1A  Not assigned      Not assigned
2        M2A  Not assigned      Not assigned
3        M3A    North York         Parkwoods
4        M4A    North York  Victoria Village
(181, 3)


After crating the table I will clean the table, removing nulls and other invalid data.

In [3]:
# cleaning the table
df1 = df
df1 = df1[~df1['PostalCode'].isnull()]  
df1.drop(df1[df1['Borough']=="Not assigned"].index,axis=0, inplace=True)
print(df1.head(5))
print(df1.shape)

  PostalCode           Borough                                Neighbourhood
3        M3A        North York                                    Parkwoods
4        M4A        North York                             Victoria Village
5        M5A  Downtown Toronto                    Regent Park, Harbourfront
6        M6A        North York             Lawrence Manor, Lawrence Heights
7        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [4]:
# merging postal codes
df2 = df1
df2 = df2.reset_index()
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']
print(df2.head(5))
print(df2.shape)

                Borough                           Neighbourhood
PostalCode                                                     
M1B         Scarborough                          Malvern, Rouge
M1C         Scarborough  Rouge Hill, Port Union, Highland Creek
M1E         Scarborough       Guildwood, Morningside, West Hill
M1G         Scarborough                                  Woburn
M1H         Scarborough                               Cedarbrae
(103, 2)


In [5]:
# Final table
df_final = df2
df_final.rename(columns={'Neighbourhood':'Neighborhood'},inplace=True)
df_final = df_final.reset_index()
print(df_final.head(15))
print(df_final.shape)

   PostalCode      Borough                                       Neighborhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...
11        M1R  Scarborough                                  Wexf

# Second Step - Mapping

In [6]:
# libraries to import
import pandas as pd
import numpy as np

In [7]:
latlon_df=pd.read_csv('http://cocl.us/Geospatial_data')
latlon_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
latlon_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_geo = pd.merge(df_final, latlon_df, on='PostalCode')
df_geo.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
CLIENT_ID = 'KSAQZZJC23YINY1EVGBP2O1FI1IWWNEAV4WTZSWW2LIWEW5M' # your Foursquare ID
CLIENT_SECRET = 'VEIFWQ4COHJI31OWZSDTTG0FWCA3NS34UI45P2PXZ5Y4VJ5K' # your Foursquare Secret
VERSION = '20180604'

In [10]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Working only with Postal Code started with M3.

In [11]:
#df_geo=df_geo[df_geo['Borough'].str.contains("East Toronto")]
df_geo=df_geo[df_geo['PostalCode'].str.contains("M1")]
df_geo.head(25)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [12]:
toronto_venues = getNearbyVenues(names=df_geo['Neighborhood'],
                                   latitudes=df_geo['Latitude'],
                                   longitudes=df_geo['Longitude']
                                  )

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge


In [13]:
print(toronto_venues.shape)
toronto_venues.head()

(94, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern, Rouge",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [14]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff, Cliffside West",4,4,4,4,4,4
Cedarbrae,9,9,9,9,9,9
"Clarks Corners, Tam O'Shanter, Sullivan",12,12,12,12,12,12
"Cliffside, Cliffcrest, Scarborough Village West",3,3,3,3,3,3
"Dorset Park, Wexford Heights, Scarborough Town Centre",8,8,8,8,8,8
"Golden Mile, Clairlea, Oakridge",9,9,9,9,9,9
"Guildwood, Morningside, West Hill",8,8,8,8,8,8
"Kennedy Park, Ionview, East Birchmount Park",4,4,4,4,4,4


In [15]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head() 

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,...,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill, Port Union, Highland Creek",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,American Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Brewery,Bus Line,...,Print Shop,Rental Car Location,Restaurant,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,Agincourt,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.000000,0.0,0.0
1,"Birch Cliff, Cliffside West",0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.000000,0.0,0.0
2,Cedarbrae,0.000000,0.111111,0.0,0.111111,0.111111,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.111111,0.0,0.0
3,"Clarks Corners, Tam O'Shanter, Sullivan",0.000000,0.000000,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.083333,0.0,0.0
4,"Cliffside, Cliffcrest, Scarborough Village West",0.333333,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.000000,0.0,0.0


In [17]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
2,Cedarbrae,Caribbean Restaurant,Bank,Gas Station,Fried Chicken Joint,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,College Stadium
3,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Bank,Pharmacy,Italian Restaurant,Chinese Restaurant,Intersection,Fast Food Restaurant,Fried Chicken Joint,Gas Station
4,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store


In [18]:
import os
from sklearn.cluster import KMeans

In [19]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2], dtype=int32)

In [20]:
#del neighborhoods_venues_sorted['Cluster Labels']
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted
#print(neighborhoods_venues_sorted.dtypes)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Latin American Restaurant,Lounge,Vietnamese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
1,1,"Birch Cliff, Cliffside West",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Clothing Store,Hakka Restaurant,Grocery Store,Gas Station,Fried Chicken Joint
2,1,Cedarbrae,Caribbean Restaurant,Bank,Gas Station,Fried Chicken Joint,Lounge,Hakka Restaurant,Bakery,Thai Restaurant,Athletics & Sports,College Stadium
3,1,"Clarks Corners, Tam O'Shanter, Sullivan",Pizza Place,Noodle House,Bank,Pharmacy,Italian Restaurant,Chinese Restaurant,Intersection,Fast Food Restaurant,Fried Chicken Joint,Gas Station
4,1,"Cliffside, Cliffcrest, Scarborough Village West",American Restaurant,Skating Rink,Motel,Chinese Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
5,1,"Dorset Park, Wexford Heights, Scarborough Town...",Indian Restaurant,Vietnamese Restaurant,Brewery,Thrift / Vintage Store,Light Rail Station,Pet Store,Chinese Restaurant,Breakfast Spot,Department Store,Athletics & Sports
6,1,"Golden Mile, Clairlea, Oakridge",Bakery,Bus Line,Bus Station,Soccer Field,Ice Cream Shop,Park,Intersection,Vietnamese Restaurant,Department Store,Discount Store
7,1,"Guildwood, Morningside, West Hill",Rental Car Location,Restaurant,Electronics Store,Intersection,Breakfast Spot,Mexican Restaurant,Medical Center,Bank,Fast Food Restaurant,Fried Chicken Joint
8,1,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Discount Store,Department Store,Coffee Shop,Vietnamese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint
9,2,"Malvern, Rouge",Fast Food Restaurant,Print Shop,Vietnamese Restaurant,Chinese Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Gas Station,Fried Chicken Joint,Electronics Store


In [21]:
toronto_merged = df_geo
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace=True)
toronto_merged["Cluster Labels"]=toronto_merged["Cluster Labels"].astype("int")
print(toronto_merged.head(20))

   PostalCode      Borough                                       Neighborhood  \
0         M1B  Scarborough                                     Malvern, Rouge   
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek   
2         M1E  Scarborough                  Guildwood, Morningside, West Hill   
3         M1G  Scarborough                                             Woburn   
4         M1H  Scarborough                                          Cedarbrae   
5         M1J  Scarborough                                Scarborough Village   
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park   
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge   
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West   
9         M1N  Scarborough                        Birch Cliff, Cliffside West   
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...   
11        M1R  Scarborough  

In [22]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                       

In [37]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors

In [71]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [72]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Pub,Theater,Breakfast Spot,Café,Hotel,Event Space,Farmers Market
54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Italian Restaurant,Hotel,Bubble Tea Shop,Café,Japanese Restaurant,Cosmetics Shop,Ramen Restaurant,Bookstore
55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Clothing Store,Restaurant,Cosmetics Shop,Cocktail Bar,American Restaurant,Gym,Creperie,Moroccan Restaurant
56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cheese Shop,Seafood Restaurant,Bakery,Cocktail Bar,Restaurant,Café,Farmers Market,Beer Bar,Park
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Department Store,Bubble Tea Shop,Burger Joint,Bar,Salad Place


In [73]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    print(label)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters